<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Neural Networks

# Only do this once per VM, otherwise you'll get multiple clones and nested directories
!git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
%cd data-science-from-scratch-Python/
!pip install import-ipynb
import import_ipynb

In [ ]:
from Chapter_04 import Vector, dot

def step_function(x: float) -> float:
  return 1.0 if x >= 0 else 0.0

def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
  """Returns 1 if the perceptron 'fires', 0 if not"""
  calculation = dot(weights, x) + bias
  return step_function(calculation)

In [ ]:
# AND Gate
and_weights = [2., 2]
and_bias = -3.

assert perceptron_output(and_weights, and_bias, [1,1]) == 1
assert perceptron_output(and_weights, and_bias, [0,1]) == 0
assert perceptron_output(and_weights, and_bias, [1,0]) == 0
assert perceptron_output(and_weights, and_bias, [0,0]) == 0

In [ ]:
# OR Gate
or_weights = [2., 2]
or_bias = -1.

assert perceptron_output(or_weights, or_bias, [1,1]) == 1
assert perceptron_output(or_weights, or_bias, [1,0]) == 1
assert perceptron_output(or_weights, or_bias, [0,1]) == 1
assert perceptron_output(or_weights, or_bias, [0,0]) == 0

In [ ]:
# NOT Gate
not_weights = [-2.]
not_bias = 1.

assert perceptron_output(not_weights, not_bias, [0]) == 1
assert perceptron_output(not_weights, not_bias, [1]) == 0

In [ ]:
# XOR gate
and_gate = min
or_gate = max
xor_gate = lambda x, y: 0 if x== y else 1

In [ ]:
import math

def sigmoid(t: float) -> float:
  return 1 / (1 + math.exp(-t))

In [ ]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
  # weights includes the bias term, inputs includes a 1
  return sigmoid(dot(weights, inputs))
  

In [ ]:
from typing import List

def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    """
    Feeds the input vector through the neural network.
    Returns the outputs of all layers (not just hte last one).
    """
    outputs: List[Vector] = []

    for layer in neural_network:
      input_with_bias = input_vector + [1] # Add a constant
      output = [neuron_output(neuron, input_with_bias) #Compute the output
                for neuron in layer]
      outputs.append(output)

      # Then the input to the next layer is the output of this one
      input_vector = output

    return outputs

    

In [ ]:
xor_network = [# hidden layer
               [[20., 20, -30],   # 'and' neuron
                [20., 20, -10]],  # 'or' neuron
               # output layer
               [[-60., 60, -30]]] # '2nd input but no 1st input' neuron

# feed_forward returns the outputs of all layers, so the [-1] gets the
# final output, and the [0] gets the value out of the resulting vector
assert 0.000 < feed_forward(xor_network, [0,0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network, [1,0])[-1][0] < 1.000
assert 0.999 < feed_forward(xor_network, [0,1])[-1][0] < 1.000
assert 0.000 < feed_forward(xor_network, [1,1])[-1][0] < 0.001

In [17]:
# Backpropagation
def sqerror_gradients(network: List[List[Vector]],
                      input_vector: Vector,
                      target_vector: Vector) -> List[List[Vector]]:
  """
  Given a neural network, an input vector, and a target vector,
  make a prediction and compute the gradient of the squared error
  loss with respect to the neuron weights.
  """
  # forward pass
  hidden_outputs, outputs = feed_forward(network, input_vector)

  # gradients with respect to output neuron pre-activation outputs
  output_deltas = [output * (1 - output) * (output - target)
                    for output, target in zip(outputs, target_vector)]
  
  # gradients with respect to output neuron weights
  output_grads = [[output_deltas[i] * hidden_output
                   for hidden_output in hidden_outputs + [1]]
                  for i, output_neuron in enumerate(network[-1])]

  # gradients with respect to hidden neuron pre-activation outputs
  hidden_deltas = [hidden_output * (1 - hidden_output) *
                   dot(output_deltas, [n[i] for n in network[-1]])
                   for i, hidden_output in enumerate(hidden_outputs)]

  # gradients with respect to hidden neuron weights
  hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
                  for i, hidden_neuron in enumerate(network[0])]
                  
  return [hidden_grads, output_grads]

In [18]:
import random
random.seed(0)

# training data
xs = [[0., 0], [0., 1], [1.,0], [1.,1]]
ys = [[0.], [1.], [1.], [0.]]

# start with random weights
network = [ # hidden layer: 2 inputs -> 2 outputs
           [[random.random() for _ in range(2 + 1)], # 1st hidden neuron
            [random.random() for _ in range(2 + 1)]], # 2nd hidden neuron
           # output layer: 2 inputs -> 1 output
           [[random.random() for _ in range(2 + 1)]] # 1st output neuron
          ]

In [20]:
from Chapter_08 import gradient_step
import tqdm

learning_rate = 1.0

for epoch in tqdm.trange(20000, desc="neural net for xor"):
  for x, y in zip(xs,ys):
    gradients = sqerror_gradients(network, x, y)

    # Take a gradient step for each neuron in each layer
    network = [[gradient_step(neuron, grad, -learning_rate)
                for neuron, grad in zip(layer, layer_grad)]
               for layer, layer_grad in zip(network, gradients)]

# check that we learned xor
assert feed_forward(network, [0,0])[-1][0] < 0.01
assert feed_forward(network, [0,1])[-1][0] > 0.99
assert feed_forward(network, [1,0])[-1][0] > 0.99
assert feed_forward(network, [1,1])[-1][0] < 0.01
print(network)

neural net for xor: 100%|██████████| 20000/20000 [00:01<00:00, 11446.84it/s]

[[[7.137127692932594, 7.136523106753526, -3.251038963015925], [5.322592618595284, 5.322277541116411, -8.151999000527617]], [[11.643129333608758, -12.30113897821493, -5.491927866215813]]]
